# importing packages

In [1]:
import numpy as np

# 3

In [2]:
summation = []
for i in range(5):
    print('day', i+1)
    x = np.random.choice([6, 8, 10, 12], p=[0.25, 0.35, 0.3, 0.1])
    print('# of customers:', x)
    d = []
    for j in range(x):
        y = np.random.choice([1, 2, 3, 4], p=[0.35, 0.45, 0.15, 0.05])
        d += [y]
    print('# of breads:', d,'\n')
    summation += [sum(d)]
    
sum(summation)/5

day 1
# of customers: 6
# of breads: [3, 2, 2, 1, 2, 1] 

day 2
# of customers: 6
# of breads: [2, 1, 2, 4, 2, 2] 

day 3
# of customers: 8
# of breads: [3, 3, 2, 1, 1, 2, 1, 2] 

day 4
# of customers: 10
# of breads: [2, 2, 2, 2, 1, 1, 1, 3, 1, 1] 

day 5
# of customers: 8
# of breads: [2, 2, 3, 2, 2, 4, 1, 2] 



14.6

# 4

A) Average of two random points distance on circle

In [3]:
import random
from math import sqrt

In [4]:
N = 1000000
distances = list()

def point_generator():
    x = random.random()
    y = sqrt(1 - (x * x))
    negative = random.randint(0, 1)
    if negative:
        y = -y
    return x, y

for i in range(N):
    x_1, y_1 = point_generator()
    x_2, y_2 = point_generator()
    
    distance = sqrt((x_2 - x_1) * (x_2 - x_1) + (y_2 - y_1) * (y_2 - y_1))
    distances.append(distance)

mean = sum(distances) / N
mean

1.021351674740949

B) General Answer

$$
average \: value \: d(\theta) = \frac{1}{\pi - 0}\int_{0}^{\pi}2sin\frac{\theta}{2}d\theta = \frac{1}{\pi}(-4cos(\frac{\theta}{2}))_0^{\pi} = \frac{4}{\pi} = 1.27
$$

# 5

A) Python

C) We can generate a random number using `np.random.choice` method. It can be seen in the code. for example:

```
traveling_probabilities = [0.4, 0.3, 0.2, 0.1]
traveling_times = [40, 60, 80, 100]
np.random.choice(traveling_times, p=traveling_probabilities, size=(1))[0]
```

In [22]:
NUMBER_OF_TRUCKS, NUMBER_OF_LOADERS = 9, 2
t, MAX_TRAVELS = 0, 100 

## States

In [23]:
LOADING_QUEUE = 1
LOADING = 2
WEIGHING_QUEUE = 3
WEIGHING = 4
TRAVELING = 5

In [24]:
traveling_probabilities = [0.4, 0.3, 0.2, 0.1]
traveling_times = [40, 60, 80, 100]

weighing_probabilities = [0.7, 0.3]
weighing_times = [12, 16]

loading_probabilities = [0.3, 0.5, 0.2]
loading_times = [5, 10, 15]

## Truck Class

In [25]:
class Truck:
    def __init__(self, plate):
        self.plate = plate
        self.spent_time = 0
        self.traveling_time = 0
        self.state = LOADING_QUEUE
        
        print()
        print(t)
        print('truck {} is just created'.format(self.plate))
        

    def traveling(self):
        self.traveling_time = np.random.choice(traveling_times, p=traveling_probabilities, size=(1))[0]
        self.spent_time = 0
        self.state = TRAVELING

        print()
        print('time', t)
        print('truck {} start traveling for {}'.format(self.plate, self.traveling_time))


    def is_ended(self):
        if self.spent_time == self.traveling_time:
            self.traveling_time = 0
            self.spent_time = 0
            
            print()
            print('time', t)
            print("truck {} ended traveling".format(self.plate))
            
            return True
        
        return False

## Scaler class

In [26]:
class Scaler:
    def __init__(self):
        self.truck = None
        self.weighing_time = 0
        self.spent_time = 0
        
        print()
        print('time', t)
        print('Scaler has just been created')
        
    
    def add_truck(self, truck):
        if truck:
            self.truck = truck
            truck.state = WEIGHING
            self.weighing_time = np.random.choice(weighing_times, p=weighing_probabilities, size=(1))[0]
            self.spent_time = 0
            
            print()
            print('time', t)
            print("truck {} is weighing for {}".format(truck.plate, self.weighing_time))
        else:
            return
        
    def is_ended(self):
        if self.spent_time == self.weighing_time:
            print()
            print('time', t)
            print("truck {} ended weighing".format(self.truck.plate))
            
            truck = self.truck
            self.weighing_time = 0
            self.spent_time = 0
            self.truck = None
            return truck
        
        return None

## Loader class

In [27]:
class Loader:
    def __init__(self, id):
        self.truck = None
        self.id = id
        self.loading_time = 0
        self.spent_time = 0
        
        print()
        print('time', t)
        print('loader {} has just been created'.format(self.id))
        
        
    def add_truck(self, truck):
        if truck:
            self.loading_time = np.random.choice(loading_times, p=loading_probabilities, size=(1))[0]
            self.spent_time = 0
            self.truck = truck
            truck.state = LOADING
        else:
            return
            
            print()
            print('time', t)
            print("loading truck {} with loader for {}".format(truck.plate, self.id, self.loading_time))
    
    
    def is_ended(self):
        if self.spent_time == self.loading_time:
            print()
            print('time', t)
            print("loader {}: loading of truck {} finished at {}".format(self.id, self.truck.plate, t))
            
            truck = self.truck
            self.truck = None
            self.spent_time = 0
            self.loading_time = 0
            return truck
        
        return None

## Initializing

In [28]:
def get_all_trucks():
    return [Truck(i) for i in range(NUMBER_OF_TRUCKS)]


def get_all_loaders():
    return [Loader(i) for i in range(NUMBER_OF_LOADERS)]


trucks = get_all_trucks()
loaders = get_all_loaders()
scaler = Scaler()



0
truck 0 is just created

0
truck 1 is just created

0
truck 2 is just created

0
truck 3 is just created

0
truck 4 is just created

0
truck 5 is just created

0
truck 6 is just created

0
truck 7 is just created

0
truck 8 is just created

time 0
loader 0 has just been created

time 0
loader 1 has just been created

time 0
Scaler has just been created


In [29]:
def get_all_trucks():
    return [Truck(i) for i in range(NUMBER_OF_TRUCKS)]


def get_all_loaders():
    return [Loader(i) for i in range(NUMBER_OF_LOADERS)]

def get_init_loading_queue():
    return [truck for truck in trucks]


trucks = get_all_trucks()
loaders = get_all_loaders()
scaler = Scaler()


loading_queue = get_init_loading_queue()
weighing_queue = list()

number_of_travels = 0

while number_of_travels <= MAX_TRAVELS:
    for truck in trucks:
        if truck.state == TRAVELING:
            truck.spent_time += 1
            if truck.is_ended():
                loading_queue.append(truck)
                truck.state = LOADING_QUEUE
                number_of_travels += 1
                
    for loader in loaders:
        if loader.truck is not None:
            loader.spent_time += 1
            ended_truck = loader.is_ended()
            if ended_truck is not None:
                weighing_queue.append(ended_truck)
                ended_truck.state = WEIGHING_QUEUE
        if loader.truck is None:
            if len(loading_queue):
                loader.add_truck(loading_queue.pop(0))
    
    if scaler.truck is not None:
        scaler.spent_time += 1
        ended_truck = scaler.is_ended()
        if ended_truck is not None:
            ended_truck.traveling()
    
    if scaler.truck is None:
        if len(weighing_queue):
            scaler.add_truck(weighing_queue.pop(0))
    t += 1



0
truck 0 is just created

0
truck 1 is just created

0
truck 2 is just created

0
truck 3 is just created

0
truck 4 is just created

0
truck 5 is just created

0
truck 6 is just created

0
truck 7 is just created

0
truck 8 is just created

time 0
loader 0 has just been created

time 0
loader 1 has just been created

time 0
Scaler has just been created

time 10
loader 0: loading of truck 0 finished at 10

time 10
loader 1: loading of truck 1 finished at 10

time 10
truck 0 is weighing for 16

time 20
loader 0: loading of truck 2 finished at 20

time 20
loader 1: loading of truck 3 finished at 20

time 25
loader 0: loading of truck 4 finished at 25

time 26
truck 0 ended weighing

time 26
truck 0 start traveling for 40

time 26
truck 1 is weighing for 12

time 30
loader 0: loading of truck 6 finished at 30

time 30
loader 1: loading of truck 5 finished at 30

time 35
loader 1: loading of truck 8 finished at 35

time 38
truck 1 ended weighing

time 38
truck 1 start traveling for 60

t